Basic Urchin Setup:

In [ ]:
#If using urchin for the first time:
pip install oursin

In [1]:
import oursin as urchin
urchin.setup()
import numpy as np
import pandas as pd
import math
#path = 'INSERT PATH OF FILE HERE'
path = '/Users/jasmineschoch/Documents/UW/VBL/Urchin/Examples/data/jasmine/'

(URN) connected to server
Login sent with ID: root


In [8]:
urchin.clear()

Loading Brain Regions:

In [2]:
#Generating overall brain image:
regions = pd.read_csv(path + 'region_colors.csv')
regions.head()
areas = regions.loc[:,"Areas"]
colors = regions.loc[:,"Color"]
urchin.ccf.set_visibility({'grey':True})
urchin.ccf.set_material({'grey':'transparent-unlit'})
urchin.ccf.set_alpha({'grey':0.1})

In [87]:
#To set just one region to be visible:
urchin.ccf.set_visibility({'VISpm':True})

In [9]:
#Generating specific regions (from Nick's paper)
#To pull apart regions, clear Urchin before running. (background must be blank)
area_colors = {}
area_vis = {}
area_material = {}
for i in range(len(areas)):
  area = areas[i] + "-lh"
  area_vis[area] = True
  area_material = 'opaque-unlit'
  area_colors[area] = (colors[i])
  area = areas[i] + "-rh"
  area_vis[area] = True
  area_material = 'opaque-unlit'
  area_colors[area] = (colors[i])
urchin.ccf.set_visibility(area_vis)
urchin.ccf.set_material(area_material) 
urchin.ccf.set_color(area_colors)

In [76]:
urchin.camera.set_rotation(45,0,135)

Loading probes:

In [83]:
probe_data = pd.read_csv(path + 'all_probes.csv')
probe_data.head()

,entry_point_rl,entry_point_ap,vertical_angle,horizontal_angle,axial_angle,distance_advanced,color,session
0,-640,1500,81.400131,172.959797,0,3800,red,1
1,-3180,-4180,74.096449,322.485452,0,3900,red,1
2,-3800,-3200,72.486409,324.053181,0,3750,red,2
3,-1800,-3500,81.772533,158.999745,0,3750,red,2
4,-1960,-2260,75.936735,152.451569,0,3800,red,3


In [84]:
av = np.load(path + 'atlas.npy')
probe_data["ccf_entry_ap"]= -probe_data["entry_point_ap"] +5400
probe_data["ccf_entry_ml"]= probe_data["entry_point_rl"] +5700
#probe_data["ccf_entry_dv"]= probe_data["ccfDV"]

probe_data["ap_rounded"] = round(probe_data["ccf_entry_ap"] / 25)
probe_data["ml_rounded"] = round(probe_data["ccf_entry_ml"] / 25) 
av = np.squeeze(av)
for i in range(len(probe_data)):
    ccfAP_rounded = int(probe_data.loc[i,"ap_rounded"])
    ccfML_rounded = int(probe_data.loc[i,"ml_rounded"])
    probe_data.loc[i,'ccf_entry_dv'] = np.where(av[ccfAP_rounded, ccfML_rounded, :] == 1)[0][0] * 25
    
def Rx(t):
    return np.array([[1, 0, 0], [0, np.cos(np.deg2rad(t)), -np.sin(np.deg2rad(t))], [0, np.sin(np.deg2rad(t)), np.cos(np.deg2rad(t))]])

def Ry(t):
    return np.array([[np.cos(np.deg2rad(t)), 0, np.sin(np.deg2rad(t))], [0, 1, 0], [-np.sin(np.deg2rad(t)), 0, np.cos(np.deg2rad(t))]])

def Rz(t):
    return np.array([[np.cos(np.deg2rad(t)), -np.sin(np.deg2rad(t)), 0], [np.sin(np.deg2rad(t)), np.cos(np.deg2rad(t)), 0], [0, 0, 1]])
for i in range(len(probe_data)):
  color = probe_data.loc[i,"color"]
  if color == "red":
    probe_data.loc[i,"color"] = "#EE204D"
  elif color == "orange":
    probe_data.loc[i,"color"] = "#FF7538"
  elif color == "light green":
    probe_data.loc[i,"color"] = "#9ACD32"
  elif color == "dark green":
    probe_data.loc[i,"color"] = "#006400"  ##
  elif color == "medium green":
    probe_data.loc[i,"color"] = "#32CD32"  ##45CEA2
  elif color == "light blue":
    probe_data.loc[i,"color"] = "#1FCECB"  ##
  elif color == "dark blue":
    probe_data.loc[i,"color"] = "#002AFF"  ##
  elif color == "purple":
    probe_data.loc[i,"color"] = "#926EAE"  ##
  elif color == "pink":
    probe_data.loc[i,"color"] = "#D6006E" ##
  elif color == "violet":
    probe_data.loc[i,"color"] = "#8F509D" ##
  else:
    probe_data.loc[i,"color"] = "#000000"
def tip(index):
    horizAngle = probe_data.loc[index,"horizontal_angle"]
    vertAngle = probe_data.loc[index,"vertical_angle"]

    p = np.array([0, 1, 0])  # example value for p

    pRot = np.dot(np.dot(Rz(horizAngle), Rx(-vertAngle)), p.T)  # use -vert because we want to go from +Y towards -Z
    pCCF = np.array([-pRot[1], -pRot[2], pRot[0]])
    ccfEntry = np.array(probe_data.loc[index,["ccf_entry_ap","ccf_entry_dv","ccf_entry_ml"]])
    tip_nick = ccfEntry + pCCF * probe_data.loc[index,"distance_advanced"]  # assuming ccfEntry and distAdvanced are defined
    probe_data.loc[index,"ml"] = tip_nick[2]
    probe_data.loc[index,"ap"] = tip_nick[0]
    probe_data.loc[index,"dv"] = tip_nick[1]
probe_data.index.map(tip)
probe_data['theta'] = probe_data.loc[:,"vertical_angle"] - 90
probe_data['phi'] = probe_data.loc[:,"horizontal_angle"] - 270

In [85]:
#Creating probes
probes = []
for i in range(len(probe_data)):
    probes.append(str(i))
urchin.probes.create(probes)

In [ ]:
#Setting probe colors
probe_colors = {}
for probe in probes:
  probe_colors[probe] = probe_data.loc[int(probe),"color"]
urchin.probes.set_color(probe_colors)
#Setting probe positions
probe_positions = {}
for probe in probes:
    probe_positions[probe] = [probe_data.loc[int(probe), "ml"], probe_data.loc[int(probe), "ap"], probe_data.loc[int(probe), "dv"]]
urchin.probes.set_position(probe_positions)
#Setting probe angles
probe_angles = {}
for probe in probes:
    probe_angles[probe] = [probe_data.loc[int(probe), "phi"], probe_data.loc[int(probe), "theta"], 0]
urchin.probes.set_angle(probe_angles)


packet queue is empty, aborting


(URN) disconnected from server
(URN) connected to server
Login sent with ID: root


packet queue is empty, aborting


(URN) disconnected from server
(URN) connected to server
Login sent with ID: root


packet queue is empty, aborting


...

(URN) disconnected from server
(URN) connected to server
Login sent with ID: root
(URN) disconnected from server
(URN) connected to server
Login sent with ID: root
(URN) disconnected from server(URN) disconnected from server

(URN) connected to server
Login sent with ID: root


packet queue is empty, aborting


(URN) disconnected from server
(URN) connected to server
Login sent with ID: root
(URN) disconnected from server(URN) disconnected from server

(URN) connected to server
Login sent with ID: root


packet queue is empty, aborting
packet queue is empty, aborting


(URN) disconnected from server
(URN) connected to server
Login sent with ID: root
(URN) disconnected from server(URN) disconnected from server

(URN) connected to server
Login sent with ID: root
(URN) disconnected from server
(URN) connected to server
Login sent with ID: root
(URN) disconnected from server
(URN) connected to server
Login sent with ID: root
(URN) disconnected from server
(URN) connected to server
Login sent with ID: root
